In [1]:
import os
os.chdir('../pipeline/src/')

In [2]:
from diag_tcn_vae import DiagTcnVAE

In [4]:
model = DiagTcnVAE.load_from_checkpoint("../../data/tmp/trained_model")

/home/jovyan/stress-curves/.venv/lib/python3.9/site-packages/pytorch_lightning/utilities/migration/utils.py:51: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.5, which is newer than your current Lightning version: v2.0.4
  rank_zero_warn(


In [6]:
from tep_data_module import TEPDataModule

In [13]:
import pandas as pd
import joblib
import constants as const

In [16]:
scaler = joblib.load("../../data/tmp/scaler")
df_train = pd.read_parquet("../../data/tmp/df_train.parquet")
df_val = pd.read_parquet("../../data/tmp/df_val.parquet")
df_test = pd.read_parquet("../../data/tmp/df_test.parquet")


In [17]:
! ls ../../data/tmp

df_test.parquet      df_train.parquet	    df_val.parquet  trained_model
df_training.parquet  df_validation.parquet  scaler


In [26]:
if any(df_train.empty, None):
    print('hi')


TypeError: any() takes exactly one argument (2 given)

In [19]:
dm = TEPDataModule(df_train=df_train, df_val=df_val, df_test=df_test, batch_size=100, input_cols=const.DATA_COLS, subsystems_map=const.SUBSYSTEM_MAP, scaler=scaler)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().